In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
import importlib
import time
import datetime
import json
import threading
from threading import Event

import shioaji as sj
import login.shioaji_login as shioaji_login
from shioaji import BidAskFOPv1
# from shioaji.constant import Action, OptionRight, StockPriceType, FuturesOrderType, FuturesOCType #選擇權下單，多匯入一個OptionRight常數
importlib.reload(shioaji_login)

import tools.globals as globals
import tools.get_snap_options as snap
import tools.message_log as message_log
import tools.get_positions as positions
import tools.cover as cover
import tools.contract as contract

In [ ]:
def update_config():
    """
    Threading function.
    每秒更新config相關參數
    
    :global param: get_simulation_time
    :global param: simulation_mode
    :global param: simulation_optionright
    :global param: simulation_quantity
    :global param: simulation_action

    :global param: get_cover_time
    :global param: cover_put_strike
    :global param: cover_call_strike
    :global param: cover_quantity

    :return: None
    """

    pre_get_simulation_time = None
    pre_simulation_mode = None
    pre_simulation_optionright = None
    pre_simulation_quantity = None
    pre_simulation_action = None

    pre_get_cover_time = None 
    pre_cover_mode = None
    pre_cover_put_strike = None 
    pre_cover_call_strike = None 
    pre_cover_quantity = None

    while(True):

        with open('config.json') as f:
            config_data = json.load(f)

            globals.get_simulation_time =  datetime.datetime.strptime(config_data['get_simulation_time'], '%H:%M:%S').time()
            globals.get_cover_time =  datetime.datetime.strptime(config_data['get_cover_time'], '%H:%M:%S').time()

            ### set simulation ###
            if config_data['simulation_mode'].lower() == "false":
                globals.simulation_mode = False
            else:
                globals.simulation_mode = True

            if config_data['simulation_optionright'].lower() == "c":
                globals.simulation_optionright = 'C'
            else:
                globals.simulation_optionright = 'P'

            if config_data['simulation_action'].lower() == "buy":
                globals.simulation_action = 'Buy'
            else:
                globals.simulation_action = 'Sell'

            globals.simulation_quantity = config_data['simulation_quantity']
            
            ### set cover ###
            if config_data['cover_mode'].lower() == "false":
                globals.cover_mode = False
            else:
                globals.cover_mode = True

            globals.cover_put_strike = config_data['cover_put_strike']
            globals.cover_call_strike = config_data['cover_call_strike']
            globals.cover_quantity = config_data['cover_quantity']

            ### detect ###
            if(pre_get_simulation_time != globals.get_simulation_time):
                print(f'Get simulation time has been set to {globals.get_simulation_time}')
                pre_get_simulation_time = globals.get_simulation_time

            if(pre_simulation_mode != globals.simulation_mode):
                print(f'Simulation_mode has been set to {globals.simulation_mode}')
                pre_simulation_mode = globals.simulation_mode

            if(pre_simulation_optionright != globals.simulation_optionright):
                print(f'Simulation_optionright has been set to {globals.simulation_optionright}')
                pre_simulation_optionright = globals.simulation_optionright

            if(pre_simulation_action != globals.simulation_action):
                print(f'Simulation_action has been set to {globals.simulation_action}')
                pre_simulation_action = globals.simulation_action

            if(pre_simulation_quantity != globals.simulation_quantity):
                print(f'Simulation_quantity has been set to {globals.simulation_quantity}')
                pre_simulation_quantity = globals.simulation_quantity

            if(pre_get_cover_time != globals.get_cover_time):
                print(f'Get cover time has been set to {globals.get_cover_time}')
                pre_get_cover_time = globals.get_cover_time

            if(pre_cover_mode != globals.cover_mode):
                print(f'Cover mode has been set to {globals.cover_mode}')
                pre_cover_mode = globals.cover_mode

            if(pre_cover_put_strike != globals.cover_put_strike):
                print(f'Cover put strike has been set to {globals.cover_put_strike}')
                pre_cover_put_strike = globals.cover_put_strike

            if(pre_cover_call_strike != globals.cover_call_strike):
                print(f'Cover call strike has been set to {globals.cover_call_strike}')
                pre_cover_call_strike = globals.cover_call_strike

            if(pre_cover_quantity != globals.cover_quantity):
                print(f'Cover quantity has been set to {globals.cover_quantity}')
                pre_cover_quantity = globals.cover_quantity

            time.sleep(1)



In [ ]:
def update_snap_options():
    """
    Threading function.
    每秒偵測,若現在時間符合get_simulation_time, 則找尋當下價平 
    且若simulation_mode為True則下模擬單
    
    :global param: get_simulation_time
    :global param: simulation_mode
    :return: None
    """

    while(True): 
        now = datetime.datetime.now()
        
        if(now.time().replace(microsecond=0) == globals.get_simulation_time):
            if globals.simulation_mode:
                snap.get_snap_options()
                snap.get_at_the_money_info()
                print("現在價平和之檔位及成交點數和: ", globals.at_the_money_code, globals.at_the_money)
                
                option_code = globals.at_the_money_code +  snap.get_option_code(globals.simulation_optionright)
                positions.place_order(globals.simulation_quantity, option_code, globals.simulation_optionright, globals.simulation_action)

        time.sleep(1)

In [ ]:
def place_cover_order():
    """
    json設定之平倉時間一到，便依照C、P設定之檔次及口數，下平倉，限價在第三跳
    # 步驟
    # 取得帳戶資訊
    # 確立合約
    # 確立order
    # 送出order

    :return: None
    """

    # globals.position = globals.api.list_positions(globals.api.futopt_account)
    # Offset.CLOSE: A closing offset (CO) order is a limit order that is executed at market close but can be placed anytime during the trading day. 
    while(True): 
        now = datetime.datetime.now()
        
        if(now.time().replace(microsecond=0) == globals.get_cover_time):
            if globals.cover_mode:
                ### 下實單平倉 ###
                contract = globals.cover_call_contract #選擇權Contract
                optionright = sj.constant.OptionRight.Call
                order = globals.api.Order(
                    action=sj.constant.Action.Buy,
                    price=globals.third_best_buy_price, #價格
                    quantity=globals.cover_quantity, #口數
                    price_type=sj.constant.StockPriceType.LMT, #MKT: 市價 LMT: 限價
                    order_type=sj.constant.FuturesOrderType.ROD,
                    octype=sj.constant.FuturesOCType.Cover, #倉別，收盤時平倉
                    OptionRight=optionright, #選擇權類型
                    account=globals.api.futopt_account #下單帳戶指定期貨帳戶
                )

                trade = globals.api.place_order(contract, order)
                print('***')
                log_msg = f'An cover order {trade} is already sent!'\n'
                print(log_msg)
                message_log.write_log(log_msg)
                print('***\n')
            
            else:
                ### 下模擬單平倉 ###

        time.sleep(1)

In [ ]:
def load_position():
    # 取得Position 
    globals.position = globals.api.list_positions(globals.api.futopt_account)

    if len(globals.position) == 0:
        print("None")

In [ ]:
def main():
    """
    Controlled whole program, make app execute.
    
    :global param: api
    """
    # initialize all global variables
    globals.initialize()

    # load_position()

    # log in
    globals.api = shioaji_login.login()
    globals.api.set_order_callback(positions.place_cb)

    # Start update config thread. All config variable will be updated every second.
    update_config_thread = threading.Thread(target = update_config)
    update_config_thread.start()
    time.sleep(1)

    update_snap_options_thread = threading.Thread(target = update_snap_options)
    update_snap_options_thread.start()
    time.sleep(1)

    place_cover_order_thread = threading.Thread(target = place_cover_order)
    place_cover_order_thread.start()
    time.sleep(1)

    snap.get_snap_options()
    snap.get_at_the_money_info()
    print("現在價平和之檔位及成交點數和: ", globals.at_the_money_code, globals.at_the_money)
    
    ### 訂閱平倉檔次之bidask ###
    cover_call_code, cover_put_code = cover.get_cover_code()
    print("put: ", cover_put_code, "call: ", cover_call_code)
    cover.subscribe_cover_code(cover_call_code, cover_put_code)
    

    # print(api.Contracts.Options.TX4)



In [ ]:
if __name__ == "__main__":
    main() 

In [ ]:
# 目前欲實現進度: 
# 1. 在get_simulation_time抓到價平後下模擬單，並在達到停損條件後(成交價or強制平倉時間)出掉
# 2. 做最佳移動掛單
# 3. 完成模擬下單
# 4. cover那邊加一個控制參數 Done

'''
positions.fill_contract(option_code)
contract_snap = globals.api.snapshots([globals.contract])
print(contract_snap)
'''